In [13]:
pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ---------------------

In [19]:
import http.client
import json
import pandas as pd
import csv
import psycopg2

In [13]:
import requests

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"1000"}

headers = {
	"x-rapidapi-key": "bc68487461mshca1d2214a6e1c46p1c3c47jsn5f4df3a84144",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


data= response.json()

#Saving into a file
filename = "PropertyRecords.json"
with open(filename,"w") as file:
    json.dump(data,file,indent=4)


10


In [7]:
#Read into a dataframe

Propertyrecord_df = pd.read_json('PropertyRecords.json')

In [9]:
Propertyrecord_df.info()
print(Propertyrecord_df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         8 non-null      float64
 1   bedrooms          7 non-null      float64
 2   squareFootage     8 non-null      float64
 3   county            10 non-null     object 
 4   propertyType      8 non-null      object 
 5   addressLine1      10 non-null     object 
 6   city              10 non-null     object 
 7   state             10 non-null     object 
 8   zipCode           10 non-null     int64  
 9   formattedAddress  10 non-null     object 
 10  yearBuilt         7 non-null      float64
 11  features          10 non-null     object 
 12  assessorID        7 non-null      object 
 13  legalDescription  7 non-null      object 
 14  subdivision       7 non-null      object 
 15  ownerOccupied     6 non-null      float64
 16  lotSize           7 non-null      float64
 17  

In [59]:
#Transformational 

#Convert columns with dictionary into string
import json

Propertyrecord_df['features']=Propertyrecord_df ['features'].apply(json.dumps)

#fill missing nd NA values
Propertyrecord_df.fillna(
    {
        'bathrooms': 0,
        'bedrooms': 0,
        'squareFootage': 0, 
        'county': "Not Available", 
        'propertyType': "unknown",
        'addressLine1': "unknown",
        'city': "unknown", 
        'state': "unknown", 
        'zipCode': "unknown", 
        'formattedAddress': "Not Available",
        'yearBuilt': 0, 
        'features': "unknown", 
        'assessorID': "unknown", 
        'legalDescription': "Not Available",
        'subdivision': "Not Available", 
        'ownerOccupied': 0, 
        'lotSize': 0, 
        'taxAssessment': "Not Available",
        'propertyTaxes': "Not Available", 
        'lastSaleDate': "Not Available", 
        'lastSalePrice': 0, 
        'owner': "unknown", 
        'id': "unknown",
        'longitude': "unknown", 
        'latitude': "unknown", 
        'zoning': "unknown", 
        'addressLine2': "Not Available"
    },
    inplace=True
)


In [63]:
print(Propertyrecord_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         10 non-null     float64
 1   bedrooms          10 non-null     float64
 2   squareFootage     10 non-null     float64
 3   county            10 non-null     object 
 4   propertyType      10 non-null     object 
 5   addressLine1      10 non-null     object 
 6   city              10 non-null     object 
 7   state             10 non-null     object 
 8   zipCode           10 non-null     int64  
 9   formattedAddress  10 non-null     object 
 10  yearBuilt         10 non-null     float64
 11  features          10 non-null     object 
 12  assessorID        10 non-null     object 
 13  legalDescription  10 non-null     object 
 14  subdivision       10 non-null     object 
 15  ownerOccupied     10 non-null     float64
 16  lotSize           10 non-null     float64
 17  

In [55]:
#create the FACT Table
fact_columns =['addressLine1','city', 'state','zipCode', 'formattedAddress',  'squareFootage', 'yearBuilt', 'bathrooms', 'bedrooms', 'lotSize', 'propertyType',
                'longitude', 'latitude']

fact_table = Propertyrecord_df[fact_columns]

#Create Location Dimension
Location_dim= Propertyrecord_df[['addressLine1','city', 'state','zipCode','county','longitude', 'latitude']].drop_duplicates().reset_index(drop=True)
Location_dim.index.name= 'location_id'

In [51]:
#Create Sales Dimension

sales_dim = Propertyrecord_df[['lastSalePrice','lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name= 'sales_id'
sales_dim.head()

,lastSalePrice,lastSaleDate
sales_id,,
0,460000.0,2022-06-21T00:00:00.000Z
1,0.0,Not Available
2,117000.0,1999-04-27T00:00:00.000Z
3,850000.0,2023-07-31T00:00:00.000Z
4,430000.0,2003-10-17T00:00:00.000Z


In [65]:
#Create Property Features Dimension
features_dim = Propertyrecord_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim.index.name ='features_id'
features_dim.head()


,features,propertyType,zoning
features_id,,,
0,"""\""\\\""{\\\\\\\""cooling\\\\\\\"": true, \\\\\\\...",Single Family,unknown
1,"""\""\\\""{}\\\""\""""",unknown,unknown
2,"""\""\\\""{\\\\\\\""architectureType\\\\\\\"": \\\\...",Single Family,R-2
3,"""\""\\\""{\\\\\\\""cooling\\\\\\\"": true, \\\\\\\...",Single Family,unknown
4,"""\""\\\""{\\\\\\\""fireplace\\\\\\\"": true, \\\\\...",Single Family,R6


In [ ]:
fact_table.to_csv('property_fact'. index=False)
location_dim.to_csv(location_dimension.csv', index=True)